In [1]:
from time import sleep

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import functions as F

spark = SparkSession \
  .builder \
  .appName("Spark Types") \
  .master("local") \
  .getOrCreate()

In [2]:
movies_df = spark.read.json("data/movies")
movies_df.show(5)

# Filters

In [3]:
# demo_literal_values
meaning_of_life_df = movies_df \
  .select(
    col("Title"),
    lit(42).alias("MOL")
  )
meaning_of_life_df.show(5, False)

In [5]:
# Логические выражения
drama_filter = movies_df.Major_Genre == "Drama"
good_rating_filter = movies_df.IMDB_Rating > 7.0

Для создания логических выражений можно использовать:
- `&` для логического 'И' / 'AND' (коньюнкция)
- `|` для логического 'ИЛИ' / 'OR' (дизьюнкция)
- `~` для логического 'НЕ' (отрицание)

In [9]:
good_drama_filter = good_rating_filter & drama_filter

In [10]:
type(good_drama_filter)

In [13]:
# Можно использовать логические выражения там же, где используются обычные колонки
# Например, в фильтрах
good_dramas_df = movies_df \
  .filter(good_drama_filter) \
  .select("Title", "Major_Genre", "IMDB_Rating")

good_dramas_df.show(5, 25)

In [12]:
good_dramas_df.explain()

In [14]:
# Также логические выражения можно использовать и в выборке колонок
movies_with_good_drama_condition_df = movies_df \
    .select(col("Title"), good_drama_filter.alias("IsItAGoodDrama"))

movies_with_good_drama_condition_df.show(5, False)

In [15]:
good_dramas_df_v2 = movies_with_good_drama_condition_df.filter("IsItAGoodDrama")

good_dramas_df_v2.show(5, False)

In [19]:
# отрицание
bad_drama_filter = ~good_drama_filter
bad_dramas = movies_df.select(col("Title"), bad_drama_filter.alias("Bad Drama"))

In [20]:
bad_dramas.explain()

In [21]:
bad_dramas.show(5, False)

# Stat num functions

In [41]:
scalled_rotten_tomatoes_rating = col("Rotten_Tomatoes_Rating") / 10
average_between_RT_and_IMDB = (scalled_rotten_tomatoes_rating + col("IMDB_Rating")) / 2

when_rating_exist = col("Rotten_Tomatoes_Rating").isNotNull() & col("IMDB_Rating").isNotNull()

movies_avg_ratings_df = movies_df \
  .select(
    col("Title"),
    average_between_RT_and_IMDB.alias("Average Rating")
  ) \
  .filter(when_rating_exist)

In [42]:
movies_avg_ratings_df.show(5, False)

In [43]:
movies_avg_ratings_df.explain()

In [49]:
# Получить статистику по таблице, например корреляцию по Пирсону [-1, 1]
rating_correlation = movies_df \
  .stat \
  .corr("IMDB_Rating", "Rotten_Tomatoes_Rating")

print(rating_correlation)

# String functions

In [57]:
movies_df \
  .select(
    initcap(col("Title")),
    upper("Director"),
    lower("Distributor")
  ) \
  .filter(movies_df["Director"].isNotNull()) \
  .show(5, False)

In [59]:
movies_df \
  .select("Title", "Director", "IMDB_Rating", "Major_Genre") \
  .filter(
    col("Title").contains("love")
  ) \
  .show(5, False)

# Regexes filtering

In [62]:
cars_df = spark.read.json("data/cars")

In [69]:
regexString = "(volkswagen|vw)"

vw_df = cars_df \
  .select(
    col("Name"),
    regexp_extract(
        col("Name"),
        regexString,
        1
    ).alias("regex_extract")
  ) \
  .filter(col("regex_extract") != "")

In [70]:
vw_df.explain()

In [71]:
vw_df.show(5, False)

In [72]:
regexString = "(volkswagen|vw)"

vw_new_name_df = vw_df \
  .select(
    col("Name"),
    regexp_replace(
        col("Name"),
        regexString,
        "Volkswagen"
    ).alias("replacement")
  )
vw_new_name_df.show(5, False)

# Задача

Выберите строки из `cars_df`, поле `Name` которых содержит либо "Ford", либо "Mercedes-Benz", либо "Volkswagen" без учета регистра. Решить задачу двумя способами:

- Через функцию `contains`
- Через регулярное выражения

In [73]:
def get_car_names():
    return ["Volkswagen", "Mercedes-Benz", "Ford"]

In [85]:
# Регулярное выражение
regexString = "|".join([name.lower() for name in get_car_names()])

cars_interest_df = cars_df \
  .select(
    col("Name"),
    regexp_extract(
        lower(col("Name")),
        regexString,
        0
    ).alias("regex_extract")
  ) \
  .filter(col("regex_extract") != "") \
  .orderBy(col("regex_extract"))

cars_interest_df.show(5, False)

In [86]:
cars_interest_df.explain()

In [88]:
# v2 - contains
car_name_filters = [
    col("Name").contains(car_name.lower())
    for car_name in ["Volkswagen", "Mercedes-Benz", "Ford"]
]

In [89]:
from functools import reduce

all_cars_filter = reduce(lambda filter1, filter2: filter1 | filter2, car_name_filters)

In [90]:
filtered_cars = cars_df.filter(all_cars_filter)

filtered_cars.show(5, False)

In [91]:
filtered_cars.explain()

# Даты

Год, состоящий из 2 символов, будет интепретироваться год после 2000, так 99 - это не 1999, а 2099. Подробнее узнать о форматах дат можно в [документации](https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html)

In [103]:
movies_with_release_dates_df = movies_df \
  .select(
    col("Title"),
    col("Release_Date"),
    to_date(
        col("Release_Date"),
        "d-MMM-yy"
    ).alias("Actual_Release")
  )

movies_with_release_dates_df.show(5, False)

In [104]:
movies_with_release_dates_df.explain()

In [106]:
# Текущая дата, текущий таймстемп, разница между датами

enriched_movies_df = movies_with_release_dates_df \
  .withColumn("Today", current_date()) \
  .withColumn("Right_Now", current_timestamp()) \
  .withColumn("Movie_Age", datediff(col("Today"), col("Actual_Release")) / 365)

enriched_movies_df.show(5, False)

In [107]:
# Выбрать пустые значения
no_release_known_df = movies_with_release_dates_df \
  .filter(col("Actual_Release").isNull())

no_release_known_df.show(5, False)

# Structures

In [118]:
# Создать структуру в поле
movies_struct_df = movies_df \
  .select(
    col("Title"),
    struct(
        col("US_Gross"),
        col("Worldwide_Gross"),
        col("US_DVD_Sales")
    ).alias("Profit")
  ) \
  .filter("US_DVD_Sales is not NULL")

movies_struct_df.show(5, False)

In [120]:
# Обращение к полям

movies_struct_field_df = movies_struct_df \
  .select(
    col("Title"),
    col("Profit") \
        .getField("US_Gross") \
        .alias("US_Profit")
  )

movies_struct_field_df.show(5, False)

In [121]:
movies_struct_field_df.explain()

In [126]:
# Получить поле через selectExpr
movies_struct_df_v2 = movies_df \
  .selectExpr("Title", "(US_Gross, Worldwide_Gross, US_DVD_Sales) as Profit") \
  .filter("US_DVD_Sales is not NULL") \
  .selectExpr("Title", "Profit.US_Gross as US_Profit")

movies_struct_df_v2.show(5, False)

In [127]:
movies_struct_df_v2.explain()

In [129]:
# Структура большой вложенности
stucture_field = """
(
  (IMDB_Rating, Rotten_Tomatoes_Rating) as Rating,
  (US_Gross, Worldwide_Gross, US_DVD_Sales) as Profit
) as Success
"""

movies_struct_df_v3 = movies_df \
  .selectExpr("Title", stucture_field)

movies_struct_df_v3.show(5, False)

In [130]:
movies_struct_field_v3 = movies_struct_df_v3 \
  .selectExpr(
    "Title",
    "Success.Rating.IMDB_Rating as IMDB"
  )

movies_struct_field_v3.show(5, False)

In [131]:
movies_struct_field_v3.explain()

# Массивы

In [141]:
regex_pattern = " |,"

movies_with_words_df = movies_df \
  .select(
    col("Title"),
    split(col("Title"), regex_pattern).alias("Title_Words"),
    split(col("Director"), regex_pattern).alias("Director_Words")
  ) \
  .filter("Director is not NULL")

movies_with_words_df.printSchema()

In [142]:
movies_with_words_df.show(10, False)

In [144]:
movies_with_words_df.explain()

In [151]:
# Операции с массивом
array_ops_df = movies_with_words_df \
  .select(
    col("Title"),
    expr("Title_Words[0]"),
    size(col("Title_Words")),
    array_contains(col("Title_Words"), "Men").alias("With 'Men' Word")
  )

array_ops_df.show(5, False)

In [152]:
array_ops_df.explain()

In [155]:
# Развернуть массив: каждому элменту массива создать новую строку

array_ops_df = movies_with_words_df \
  .select(
    col("Title"),
    explode(col("Title_Words"))
  )

array_ops_df.show(10, False)